### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = '/home/thiago/tcc_ufrj/PRE_PROCESSING'

## Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

conteudo_da_pagina = navegador.page_source
site = BeautifulSoup(conteudo_da_pagina, 'html.parser')
routes = site.findAll('tr')

list_rotas_pendentes = []
list_rotas_finalizadas = []
lista_rotas = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        rotas_pendentes = route.find('span', attrs={'class': 'text-danger'})    
        link_rotas_pendentes = route.find('a')        
        lista_rotas.append([PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']])
        # list_rotas_pendentes.append([PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']])
        print (f"{rotas_pendentes.text}: {PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']}")

    else:         
        link_rotas_finalizadas = route.find('a')        
        lista_rotas.append([PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        # list_rotas_finalizadas.append([PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']}")
time.sleep(3)
navegador.close()

#len(list_rotas_pendentes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_rotas_finalizadas) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Acessando a telas para efetuar os downloads - Renomeando os arquivo adicionando o nome dos usuários - Movendo para a pasta PRE_PROCESSING

In [ ]:
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in lista_rotas:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_conteudo_da_pagina = navegador.page_source
    site_user = BeautifulSoup(user_conteudo_da_pagina, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)        
        print(f'Rota PENDENTE: {list_route} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(nome_usuario)

    else:
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)
        print(f'Rota FINALIZADA: {list_route} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(nome_usuario)
time.sleep(3)
navegador.close()

In [ ]:
## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        novo_nome = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, novo_nome))

arquivos_para_renomear_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
arquivos_para_renomear_gpx

for user, arquivo_para_renomear_gpx in zip(users, arquivos_para_renomear_gpx):
    if os.listdir(DOWNLOADS):
        caminho_antigo = os.path.join(DOWNLOADS, arquivo_para_renomear_gpx)        
        novo_nome = f"{arquivo_para_renomear_gpx.replace('.gpx', '')}__{user}.gpx"        
        caminho_novo = os.path.join(DOWNLOADS,novo_nome)        
        #print(caminho_novo)
        os.rename(caminho_antigo, caminho_novo)

In [ ]:
# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
arquivos_para_pre_processamento = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for arquivo_para_pre_processamento in arquivos_para_pre_processamento:
    caminho_origem = os.path.join(DOWNLOADS, arquivo_para_pre_processamento)
    caminho_destino = os.path.join(PRE_PROCESSING, arquivo_para_pre_processamento)
    try:
        os.rename(caminho_origem,caminho_destino)
        print(f"Arquivo '{arquivo_para_pre_processamento}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{arquivo_para_pre_processamento}': {e}.")

## 1ª Função. Responsável por fazer o download das rotas diretamente site, renomear os arquivos e os mover para a pasta de pré processamento - Essa execução deve ser executada de uma única vez, pois pode haver alguma falha no momento de renomear os arquivos.

In [1]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIXO_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSAMENTO = '/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO'

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

conteudo_da_pagina = navegador.page_source
site = BeautifulSoup(conteudo_da_pagina, 'html.parser')
rotas = site.findAll('tr')

lista_rotas_pendentes = []
lista_rotas_finalizadas = []
lista_rotas = []

for rota in rotas:
    if rota.find('span', attrs={'class': 'text-danger'}):
        rotas_pendentes = rota.find('span', attrs={'class': 'text-danger'})    
        link_rotas_pendentes = rota.find('a')        
        lista_rotas.append([PREFIXO_URL_DOWNLOAD+link_rotas_pendentes['href']])
        # lista_rotas_pendentes.append([PREFIX_URL_DOWNLOAD+link_rotas_pendentes['href']])
        print (f"{rotas_pendentes.text}: {PREFIXO_URL_DOWNLOAD+link_rotas_pendentes['href']}")

    else:         
        link_rotas_finalizadas = rota.find('a')        
        lista_rotas.append([PREFIXO_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        # lista_rotas_finalizadas.append([PREFIX_URL_DOWNLOAD+link_rotas_finalizadas['href']])
        print(f"FINISHED: {PREFIXO_URL_DOWNLOAD+link_rotas_finalizadas['href']}")
time.sleep(3)
navegador.close()

#len(list_rotas_pendentes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_rotas_finalizadas) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)


## Baixando os arquivos 
navegador = webdriver.Chrome(service=servico)
usuarios = []
for lista_rota in lista_rotas:
    time.sleep(3)
    url = lista_rota[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    conteudo_pagina_download = navegador.page_source
    pagina_usuario = BeautifulSoup(conteudo_pagina_download, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in pagina_usuario):
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)        
        print(f'Rota PENDENTE: {lista_rota} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        usuarios.append(nome_usuario)

    else:
        tb_nome_usuario = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        nome_usuario = tb_nome_usuario.text
        nome_usuario = re.sub(r'\s|\.|\(|\)','_',nome_usuario)
        print(f'Rota FINALIZADA: {lista_rota} Usuário: {nome_usuario}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        usuarios.append(nome_usuario)
time.sleep(5)
navegador.close()


## Corrigindo o nome dos arquivos conforme seus usuários
arquivos_para_renomear = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for arquivo_para_renomear in arquivos_para_renomear:
        novo_nome = arquivo_para_renomear.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, arquivo_para_renomear), os.path.join(DOWNLOADS, novo_nome))

arquivos_para_renomear_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
arquivos_para_renomear_gpx

for usuario, arquivo_para_renomear_gpx in zip(usuarios, arquivos_para_renomear_gpx):        
    caminho_antigo = os.path.join(DOWNLOADS, arquivo_para_renomear_gpx)
    novo_nome = f"{arquivo_para_renomear_gpx.replace('.gpx', '')}__{usuario}.gpx"        
    caminho_novo = os.path.join(DOWNLOADS,novo_nome)            
    os.rename(caminho_antigo, caminho_novo)


# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
arquivos_para_pre_processamento = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for arquivo_para_pre_processamento in arquivos_para_pre_processamento:
    caminho_origem = os.path.join(DOWNLOADS, arquivo_para_pre_processamento)
    caminho_destino = os.path.join(PRE_PROCESSAMENTO, arquivo_para_pre_processamento)
    try:
        os.rename(caminho_origem,caminho_destino)
        print(f"Arquivo '{arquivo_para_pre_processamento}' movido para a pasta {PRE_PROCESSAMENTO}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{arquivo_para_pre_processamento}': {e}.")

PENDING: https://www.openstreetmap.org/user/dragonpilot/traces/10086756
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086755
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086754
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086753
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086752
PENDING: https://www.openstreetmap.org/user/%D0%9C%D0%B8%D1%85%D0%B0%D0%B8%D0%BB%20%D0%A0%D0%B0%D0%B7%D0%B3%D0%BE%D0%B2%D0%BE%D1%80%D0%BE%D0%B2/traces/10086751
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086749
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086748
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086747
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/10086746
PENDING: https://www.openstreetmap.org/user/%D0%9C%D0%B8%D1%85%D0%B0%D0%B8%D0%BB%20%D0%A0%D0%B0%D0%B7%D0%B3%D0%BE%D0%B2%D0%BE%D1%80%D0%BE%D0%B2/traces/10086745
PENDING: https://www.openstreetmap.org/u

## 2ª Função - Lendo arquivos pasta de Pré Processamento - Convertendo os arquivos .gpx em .csv - Levando os arquivos .csv para a camada bronze do datalake

In [3]:
# pip install gpxpy
# pip install minio
import gpxpy 
import gpxpy.gpx 
import pandas as pd
import os
from minio import Minio
from minio.error import S3Error
from geopy.geocoders import Nominatim


PRE_PROCESSAMENTO = '/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO'
CAMADA_BRONZE = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [4]:
## Criando arquivos CSV a partir dos arquivos .gpx, após a criação do CSV o .gpx correspondente é excluído
arquivos_pre_processamento = [arquivo for arquivo in os.listdir(PRE_PROCESSAMENTO) if arquivo.endswith(".gpx")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .gpx
for arquivo_pre_processamento in arquivos_pre_processamento:
    caminho_arquivo = os.path.join(PRE_PROCESSAMENTO, arquivo_pre_processamento) # --> Crie o caminho completo para o arquivo GPX    

In [6]:
with open('/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO/10086735__Routes_from_dragonpilot_2023_08_15__TOYOTA_RAV4_HYBRID_2019__.gpx', 'r', encoding='utf-8') as arquivo_gpx:
    gpx = gpxpy.parse(arquivo_gpx)
    

GPX(tracks=[GPXTrack(name='2023-09-10T14:40:32.072553', segments=[GPXTrackSegment(points=[...])])])

In [14]:
info_rota = [] #--> Lista que servirá de apoio para converter o arquivo .gpx em uma lista
for trilha in gpx.tracks:
    for segmento in trilha.segments:
        for ponto in segmento.points:            
            #geolocator = Nominatim(user_agent="geoapiExercises")
            #latitude = ponto.latitude
            #longitude = ponto.longitude

            #address = location.raw['address']
            #city = address.get('city', '')
            #state = address.get('state', '')
            #country = address.get('country', '')

            #location = geolocator.reverse(f'{latitude},{longitude}')
            #address = location.raw['address']
            #cidade = address.get('suburb')
            #estado = address.get('state')
            #pais = address.get('country_code')


            info_rota.append({
                'latitude': ponto.latitude,
                'longitude': ponto.longitude,
                'elevacao' : ponto.elevation,
                'time_point' : ponto.time
                #'cidade': cidade,
                #'estado': estado,
                #'pais': pais
            })
arquivo_pre_processamento_csv = arquivo_pre_processamento.replace('.gpx','.csv')
info_rota_df = pd.DataFrame(info_rota)
info_rota_df.to_csv(f'{PRE_PROCESSAMENTO}/{arquivo_pre_processamento_csv}', index=False)        
## os.remove(caminho_arquivo)

In [16]:
df = pd.read_csv(r'/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO/10086744__Routes_from_sunnypilot_2023_08_07-dev__CHEVROLET_BOLT_EUV_2022__.csv')
df

,latitude,longitude,elevacao,time_point
0,24.811010,121.200411,274.073,2023-09-10 14:40:32+00:00
1,24.810956,121.200408,274.013,2023-09-10 14:40:32.200000+00:00
2,24.810902,121.200405,273.962,2023-09-10 14:40:32.400000+00:00
3,24.810875,121.200404,273.956,2023-09-10 14:40:32.500000+00:00
4,24.810848,121.200402,273.953,2023-09-10 14:40:32.600000+00:00
...,...,...,...,...
1495,24.806944,121.158126,173.124,2023-09-10 14:43:35.200000+00:00
1496,24.806953,121.158066,172.967,2023-09-10 14:43:35.400000+00:00
1497,24.806962,121.158006,172.694,2023-09-10 14:43:35.600000+00:00
1498,24.806971,121.157946,172.501,2023-09-10 14:43:35.800000+00:00


In [23]:
num_df = len(df)
num_df

1500

In [17]:
df['time_point'] = pd.to_datetime(df['time_point'], format='%Y-%m-%d %H:%M:%S.%f%z', errors='coerce')
df['data'] = df['time_point'].dt.date
df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
df = df.drop(columns=['time_point'])

In [44]:
Latitude = df['latitude']
Longitude = df['longitude']
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(f'{Latitude},{Longitude}') 


address = location.raw['address']
cidade = address.get('suburb')
estado = address.get('ISO3166-2-lvl4')
pais = address.get('country_code')

estado = estado[-2:]
pais = pais.upper()

print(f"Cidade: {cidade}")
print(f"Estado (abreviado): {estado[-2:]}")
print(f"País: {pais}")

replic_id_rota = df['id_rota']
replic_nome_usuario = df['nome_usuario']
#
#ordenacao_df = ['id_rota', 'nome_usuario', 'latitude', 'longitude', 'elevacao', 'data', 'hora', 'cidade', 'estado', 'pais']
#

ValueError: Must be a coordinate pair or Point

In [25]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
#Latitude = "45.24559097710776"
#Longitude = "-122.79781984274484"
location = geolocator.reverse(f'{Latitude},{Longitude}') 
print(location)

Robert Hugh Baldock Freeway, Fargo, Marion County, Oregon, 97002, United States


In [ ]:
repeated_id = np.tile(id, num_df)
repeated_user = np.tile(username, num_df)

In [ ]:
# Reordenando as colunas
desired_order = ['id', 'username', 'latitude', 'longitude', 'elevacao', 'data', 'hora']
df = df.reindex(columns=desired_order)

In [ ]:
df.head()

In [ ]:
df.to_csv('arquivo.csv', index=False, sep=';')

In [ ]:
## Executando de uma unica vez ##
# pip install gpxpy
# pip install minio
import gpxpy 
import gpxpy.gpx 
import pandas as pd
import os
from minio import Minio
from minio.error import S3Error
from geopy.geocoders import Nominatim


PRE_PROCESSAMENTO = '/home/thiago/tcc_ufrj/PRE_PROCESSAMENTO'
CAMADA_BRONZE = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))


## Criando arquivos CSV a partir dos arquivos .gpx, após a criação do CSV o .gpx correspondente é excluído
arquivos_pre_processamento = [arquivo for arquivo in os.listdir(PRE_PROCESSAMENTO) if arquivo.endswith(".gpx")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .gpx
for arquivo_pre_processamento in arquivos_pre_processamento:
    caminho_arquivo = os.path.join(PRE_PROCESSAMENTO, arquivo_pre_processamento) # --> Crie o caminho completo para o arquivo GPX

    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo_gpx:
            gpx = gpxpy.parse(arquivo_gpx)

        info_rota = [] #--> Lista que servirá de apoio para converter o arquivo .gpx em uma lista
        for trilha in gpx.tracks:
            for segmento in trilha.segments:
                for ponto in segmento.points:
                    
                    geolocator = Nominatim(user_agent="geoapiExercises")
                    latitude = ponto.latitude
                    longitude = ponto.longitude
                    location = geolocator.reverse(f'{Latitude},{Longitude}') 


                    address = location.raw['address']
                    city = address.get('city', '')
                    state = address.get('state', '')
                    country = address.get('country', '')

                    location = geolocator.reverse(f'{latitude},{longitude}')
                    address = location.raw['address']
                    cidade = address.get('suburb')
                    estado = address.get('state')
                    pais = address.get('country_code')


                    info_rota.append({
                        'latitude': ponto.latitude,
                        'longitude': ponto.longitude,
                        'elevacao' : ponto.elevation,
                        'time_point' : ponto.time,
                        'cidade': cidade,
                        'estado': estado,
                        'pais': pais
                    })
        arquivo_pre_processamento_csv = arquivo_pre_processamento.replace('.gpx','.csv')
        info_rota_df = pd.DataFrame(info_rota)
        info_rota_df.to_csv(f'{PRE_PROCESSAMENTO}/{arquivo_pre_processamento_csv}', index=False)        
        os.remove(caminho_arquivo)    
    except Exception as e:
        print(f"Erro ao converter o '{arquivo_pre_processamento}': {e}.") #--> Exibindo o erro
        os.remove(caminho_arquivo)
        print(f"Excluindo o arquivo defeituoso: {arquivo_pre_processamento}")



arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSAMENTO) if files.endswith(".csv")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .csv
for nome_arquivo in arquivos_para_datalake: #--> Iterando sobre cada item da lista
    caminho_pre_proc = os.path.join(PRE_PROCESSAMENTO, nome_arquivo) #--> Criando o caminho completo para o arquivo .csv
    if os.path.isfile(caminho_pre_proc): #--> Verificando se o caminho especificado está apontando para um arquivo válido no sistema de arquivos.
        try:
            minioclient.fput_object(CAMADA_BRONZE, nome_arquivo, caminho_pre_proc) #--> Usando o cliente Minio para enviar o arquivo para o bucket especificado (CAMADA_BRONZE)
            print(f"Arquivo {nome_arquivo} enviado com sucesso para o bucket.") #--> Exibindo a mensagem de sucesso após o upload dos arquivos para o bucket.
            os.remove(caminho_pre_proc) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta PRE_PROCESSAMENTO
        except S3Error as e: #--> Capturando qualquer erro que porventura ocorra
            print(f"Erro ao enviar o arquivo: {nome_arquivo} -> Erro: {e}") #--> Exibindo o erro

In [ ]:
PADRAO_REGEX = r'(.*?)__'
padrao_encontrado = re.findall(PADRAO_REGEX, nome_arquivo)
id_rota = padrao_encontrado[0]
nome_usuario = padrao_encontrado[-1]
padrao_encontrado

In [ ]:
arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSAMENTO) if files.endswith(".csv")]
#arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSAMENTO) if files.endswith(".parquet")]
len(arquivos_para_datalake)

In [ ]:
## Aqui os arquivos com a extensão .csv saem da pasta PRE_PROCESSING para a primeira camada (bronze) no MinIO
#arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".csv")]
arquivos_para_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".parquet")]
for nome_arquivo in arquivos_para_datalake:
    caminho_pre_proc = os.path.join(PRE_PROCESSING, nome_arquivo)    
    if os.path.isfile(caminho_pre_proc):
        try:
            minioclient.fput_object(BRONZE_LAYER, nome_arquivo, caminho_pre_proc)
            print(f"Arquivo {nome_arquivo} enviado com sucesso para o bucket.")
            os.remove(caminho_pre_proc) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {nome_arquivo} -> Erro: {e}")     

In [ ]:
with open('PRE_PROCESSING/9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx', 'r', encoding='utf-8') as arquivo_gpx:
    gpx = gpxpy.parse(arquivo_gpx)

In [ ]:
arquivo_gpx = '9632229__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx'

In [ ]:
arquivo_gpx = str(arquivo_gpx)

In [ ]:
type(arquivo_gpx)

In [ ]:
with open(arquivo_gpx, 'r', encoding='utf-8') as arquivo_gpx:
    gpx = gpxpy.parse(arquivo_gpx)

In [ ]:
gpx

In [ ]:
type(gpx)

In [ ]:
# Para retornar a quantidade de pontos (lat+lon+ele) em um arquivo arquivo '.gpx' podemos usar get
# Documentação: https://pypi.org/project/gpxpy/
gpx.get_track_points_no()

In [ ]:
# Para retornar a faixa de altitude, a fim de se obter extremos de maior elevação e menor elevação no trajeto percorrido - os valores apresentados são em metros acima do nível do mar
gpx.get_elevation_extremes()

In [ ]:
# Não endendi para que isso serve
gpx.get_uphill_downhill()

In [ ]:
# Obtendo o nome do criador da rota - Entretanto no nosso caso quando usamos esse comando temos uma informação generica - Foi necessário obter o nome do criador da rota com o WebScraping
creator = gpx.creator
creator

In [ ]:
# Para exibir o conteudo do arquivo .gpx em formato xml
print(gpx.to_xml()[:1000])

In [ ]:
# É possivel verificar quantas rotas/trilhas nosso arquivo .gpx possui
len(gpx.tracks)

In [ ]:
# Como no nosso exemplo acima só temos 1 rota/trilha efetuada, podemos acessar por meio de indice no python, 
# Nesse caso se passa o valor [0] para ter uma precisão exata da rota em que estamos trabalhando
gpx.tracks[0]

In [ ]:
# Agora vamos acessar os segmentos da nossa rota/trilha
gpx.tracks[0].segments[0]

In [ ]:
# Agora podemos acessar os pontos de dados individuais dentro do nosso gpx acessando a matriz de pontos. Aqui tambem podemo0s ver o nome das propriedades do arquivo como elevation e time
gpx.tracks[0].segments[0].points[:10]

In [ ]:
# Aqui acessamos as tres camadas: Track - Segments e Point e o passamos para um array em forma de dicionário. Note que podemos acessar cada atributo. No meu caso estou pegando latitude, Longitude, Elevação e a hora de cada ponto (entenda-se como ponto Lat+Long+Ele)
info_rota = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            info_rota.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevacao' : point.elevation,
                'time_point' : point.time
            })

In [ ]:
# Exibindo somente os 3 primeiros resultados
info_rota[:3]

In [ ]:
type(info_rota)

In [ ]:
# Aqui podemos transformar essa informação em um dataframe com a biblioteca pandas
info_rota_df = pd.DataFrame(info_rota)
info_rota_df.head()

In [ ]:
type(info_rota_df)

In [ ]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_csv('pandas_info_rota.csv', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

In [ ]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_parquet('pd_parquet_info_rota.parquet', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

## 3ª Função - Lendo o bucket e transformando os arquivos encontrados lá em DF Pandas

In [ ]:
# pip install minio
# pip install pandas
# pip install numpy

from minio import Minio
from minio.error import S3Error
from io import StringIO, BytesIO
import pandas as pd
import re
import numpy as np

CAMADA_BRONZE = 'bronze'
CAMADA_SILVER = 'silver'
PADRAO_REGEX = r'(.*?)__'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
arquivos_rotas_gpx_csv


In [ ]:
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
    try:

        ### OBTENDO O ARQUIVO E O CONVERTENDO EM UM DF PANDAS ###
        obj_rota_csv = minioclient.get_object(CAMADA_BRONZE, arquivo_rotas_gpx_csv.object_name)        
        csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
        arquivo_csv = StringIO(csv_decod)
        df = pd.read_csv(arquivo_csv)

        #### SEPARANDO A INFORMAÇÃO DE DATA E HORA EM 2 COLUNAS SEPARADAS ###
        df['time_point'] = pd.to_datetime(df['time_point'], format='%Y-%m-%d %H:%M:%S.%f%z', errors='coerce')
        df['data'] = df['time_point'].dt.date
        df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
        df = df.drop(columns=['time_point'])
    except S3Error as e:
        print(f"Erro ao alterar a data do arquivo: {nome_arquivo}. Erro: {e}")

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
arquivos_rotas_gpx_csv

In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_BRONZE) if arquivo_gpx.object_name.endswith(".csv")]
for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:

    ### OBTENDO O ARQUIVO E O CONVERTENDO EM UM DF PANDAS ###
    obj_rota_csv = minioclient.get_object(CAMADA_BRONZE, arquivo_rotas_gpx_csv.object_name)        
    csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string    
    arquivo_csv = StringIO(csv_decod)
    df = pd.read_csv(arquivo_csv)

    ### SEPARANDO A INFORMAÇÃO DE DATA E HORA EM 2 COLUNAS SEPARADAS ###
    df['time_point'] = pd.to_datetime(df['time_point'])
    df['data'] = df['time_point'].dt.date
    df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
    df = df.drop(columns=['time_point']) 

    ### USANDO O REGEX PARA PEGAR O ID DA RODA E O NOME DO USUÁRIO (DADOS PRESENTES NO NOME DO ARQUIVO) ###
    nome_arquivo = arquivo_rotas_gpx_csv.object_name        
    padrao_encontrado = re.findall(PADRAO_REGEX, nome_arquivo)
    id_rota = padrao_encontrado[0]
    nome_usuario = padrao_encontrado[-1]
    num_df = len(df)
    replic_id_rota = np.tile(id_rota, num_df)
    replic_nome_usuario = np.tile(nome_usuario, num_df)
    df['id_rota'] = replic_id_rota
    df['nome_usuario'] = replic_nome_usuario
    ordenacao_df = ['id_rota', 'nome_usuario', 'latitude', 'longitude', 'elevacao', 'data', 'hora']
    df = df.reindex(columns=ordenacao_df)

    try:
        csv_bytes = df.to_csv(index=False).encode('utf-8') 
        csv_buffer = BytesIO(csv_bytes)
        minioclient.put_object(
                            CAMADA_SILVER,
                            nome_arquivo,
                            data=csv_buffer,
                            length=len(csv_bytes),
                            content_type='application/csv')

        #minioclient.remove_object(CAMADA_BRONZE, nome_arquivo)        

    except S3Error as e:
        print(f"Erro ao enviar o arquivo: {nome_arquivo} para a [camada silver]. Erro: {e}")

In [ ]:
destino_csv_silver

In [ ]:
df.head()

In [ ]:
blob_rota_csv = minioclient.get_object(BRONZE_LAYER, '9781413__Routes_from_dragonpilot_2023_07_05__TOYOTA_COROLLA_TSS2_2019__.csv')
rota_csv = blob_rota_csv.data
rota_csv = rota_csv.decode("ISO-8859-1")
pre_df = StringIO(rota_csv)
df = pd.read_csv(pre_df)

In [ ]:
df.head()

In [ ]:
df['time_point'] = pd.to_datetime(df['time_point'])
df['data'] = df['time_point'].dt.date
df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
df = df.drop(columns=['time_point'])


In [ ]:
df.head()

In [ ]:
import re
import numpy as np


text = "9781892__Routes_from_sunnypilot_0_9_3_1-release__HONDA_ACCORD_2018__.gpx"

# Definindo o padrão de regex
pattern = r'(.*?)__'

# Encontrando todas as correspondências no texto
matches = re.findall(pattern, text)

# Extraindo o nome do carro
username = matches[-1]
id = matches[0]

print(f'Id: {id} - username: {username}')


In [ ]:
num_df = len(df)
num_df

In [ ]:
repeated_id = np.tile(id, num_df)
repeated_user = np.tile(username, num_df)


In [ ]:
df['id'] = repeated_id
df['username'] = repeated_user


In [ ]:
# Reordenando as colunas
desired_order = ['id', 'username', 'latitude', 'longitude', 'elevacao', 'data', 'hora']
df = df.reindex(columns=desired_order)

In [ ]:
df.head()

In [ ]:
df.to_csv('arquivo.csv', index=False, sep=';')

# Escrevendo os arquivos no banco de dados Postgres

In [ ]:
# pip install psycopg2
import psycopg2
import os
from minio import Minio
import io

CAMADA_SILVER = 'silver'


minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

db_config = {
'host': 'localhost',
'database': 'gold-saint',
'user': 'postgres',
'password': 'postgres',
}

copy_sql = """
    COPY tb_gpx_full (id_rota, nome_usuario, latitude, longitude, elevacao, data_rota, hora_rota)
    FROM stdin WITH CSV HEADER DELIMITER as ','
"""


In [ ]:
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_SILVER) if arquivo_gpx.object_name.endswith(".csv")]
if not arquivos_rotas_gpx_csv:
    print("Não existem arquivos CSV no bucket. Nenhuma carga de dados será executada.")
else:
    print(f"Existem {len(arquivos_rotas_gpx_csv)} arquivos no Bucket")

In [ ]:
try:
    # Lista todos os arquivos na camada "silver" do Minio que têm extensão .csv
    arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(CAMADA_SILVER) if arquivo_gpx.object_name.endswith(".csv")]
    
    # Verifica se há arquivos no bucket antes de continuar
    if not arquivos_rotas_gpx_csv:
        print("Não existem arquivos CSV no bucket. Nenhuma carga de dados será executada.")

    else:
        # Conexão com o banco de dados PostgreSQL
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # Itera sobre cada arquivo CSV encontrado no Minio
        for arquivo_rotas_gpx_csv in arquivos_rotas_gpx_csv:
            # Obtém o objeto do arquivo CSV do Minio  
            obj_rota_csv = minioclient.get_object(CAMADA_SILVER, arquivo_rotas_gpx_csv.object_name)            

            # Decodifica os dados do arquivo CSV de bytes para string
            csv_decod = obj_rota_csv.data.decode('utf-8')  # Convertendo bytes para string            

            # Usa io.StringIO para criar um objeto de arquivo legível a partir da string CSV
            with io.StringIO(csv_decod) as file:        

                # Executa o comando COPY para inserir os dados no banco de dados PostgreSQL
                cursor.copy_expert(sql=copy_sql, file=file)

            # Commit para salvar as alterações no banco de dados    
            conn.commit()        

            # Remove o arquivo do bucket "silver" no Minio após ser processado
            # minioclient.remove_object(CAMADA_SILVER, arquivo_rotas_gpx_csv.object_name) 

        # Fecha a conexão com o banco de dados PostgreSQL
        conn.close()

except Exception as e:
    # Em caso de erro, imprime a mensagem de erro
    print(f"Erro: {str(e)}")

# Usando o Pandas e o Spark para leitura dos dados no Postgres

In [7]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession



In [8]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "45.24559097710776"
Longitude = "-122.79781984274484"
location = geolocator.reverse(f'{Latitude},{Longitude}') 
print(location)

Robert Hugh Baldock Freeway, Fargo, Marion County, Oregon, 97002, United States


In [10]:
address = location.raw['address']
address

{'road': 'Robert Hugh Baldock Freeway',
 'suburb': 'Fargo',
 'county': 'Marion County',
 'state': 'Oregon',
 'ISO3166-2-lvl4': 'US-OR',
 'postcode': '97002',
 'country': 'United States',
 'country_code': 'us'}

In [19]:
address = location.raw['address']
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')

In [21]:
address

{'road': 'Robert Hugh Baldock Freeway',
 'suburb': 'Fargo',
 'county': 'Marion County',
 'state': 'Oregon',
 'ISO3166-2-lvl4': 'US-OR',
 'postcode': '97002',
 'country': 'United States',
 'country_code': 'us'}

In [22]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "45.24559097710776"
Longitude = "-122.79781984274484"
location = geolocator.reverse(f'{Latitude},{Longitude}') 
print(location)

address = location.raw['address']
cidade = address.get('suburb')
estado = address.get('ISO3166-2-lvl4')
pais = address.get('country_code')

In [23]:
print("Cidade:", cidade)
print("Estado (abreviado):", estado)
print("País:", pais)

Cidade: Fargo
Estado (abreviado): US-OR
País: us


In [2]:
db_config = {
    'host': 'localhost',
    'database': 'gold-saint',
    'user': 'postgres',
    'password': 'postgres',
}


In [3]:
sql_query = "SELECT * FROM tb_gpx_full"

In [7]:
conn = psycopg2.connect(**db_config)

df_pandas = pd.read_sql_query(sql_query, conn)

conn.close()

df_pandas


/tmp/ipykernel_1552/1288466255.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pandas = pd.read_sql_query(sql_query, conn)


,id_rota,nome_usuario,latitude,longitude,elevacao,data_rota,hora_rota
0,10070634,HONDA_ACCORD_HYBRID_2018,45.24559097710776,-122.79781984274484,35.005,2023-09-09,22:06:36
1,10070634,HONDA_ACCORD_HYBRID_2018,45.24561488243122,-122.7978000436472,35.005,2023-09-09,22:06:36
2,10070634,HONDA_ACCORD_HYBRID_2018,45.24563834363882,-122.79778075805474,34.961,2023-09-09,22:06:36
3,10070634,HONDA_ACCORD_HYBRID_2018,45.24566171199957,-122.79776132359484,34.961,2023-09-09,22:06:36
4,10070634,HONDA_ACCORD_HYBRID_2018,45.24568465046427,-122.79774231123491,34.92,2023-09-09,22:06:36
...,...,...,...,...,...,...,...
2386219,10075036,TOYOTA_COROLLA_2017,33.9952147848659,-85.98871431280011,152.481,2023-09-09,03:33:17
2386220,10075036,TOYOTA_COROLLA_2017,33.99521479131348,-85.98871417193261,152.503,2023-09-09,03:33:17
2386221,10075036,TOYOTA_COROLLA_2017,33.99521479225636,-85.98871401635428,152.548,2023-09-09,03:33:17
2386222,10075036,TOYOTA_COROLLA_2017,33.99521479728368,-85.98871386889377,152.56300000000002,2023-09-09,03:33:17


In [25]:
def obter_info_localizacao(df):
    geolocator = Nominatim(user_agent="geoapiExercises")
    latitudes = df['latitude'].astype(str)
    longitudes = df['longitude'].astype(str)

    def obter_localizacao(latitude, longitude):
        location = geolocator.reverse(f'{latitude},{longitude}')
        address = location.raw['address']
        cidade = address.get('suburb')
        estado = address.get('state')
        pais = address.get('country_code')
        return cidade, estado, pais
    df[['cidade', 'estado', 'pais']] = zip(*[obter_localizacao(lat, lon) for lat, lon in zip(latitudes, longitudes)])
    return df

In [ ]:
df_resultante = obter_info_localizacao(df_pandas)
df_resultante

In [ ]:
df_spark = spark.createDataFrame(df_pandas)

In [28]:
appname = 'tcc-project'
master = 'local'
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

In [9]:
spark

## Códigos não mais usados

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for nome_arquivos in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, nome_arquivos)
#     destiny_path = os.path.join(DATALAKE_STAGE, nome_arquivos)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {nome_arquivos} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     caminho_pre_proc = os.path.join(DATALAKE_STAGE, item)
#     print(caminho_pre_proc)
# 
#     if os.path.isfile(caminho_pre_proc):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, caminho_pre_proc)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP ## #
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()

In [ ]:
# ## ESTE TRECHO TEM A FINALIDADE DE BAIXAR OS ARQUIVOS ## #
#list_rotas_finalizadas
#list_rotas_pendentes

# //*[@id="content"]/div[2]/div/table/tbody/tr[6]/td - finalizado
# //*[@id="content"]/div[2]/div/span = PENDENTE
# //*[@id="content"]/div[2]/div/table/tbody/tr[4]/td - pendente

# ## Trecho descontinuado -- Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
# navegador = webdriver.Chrome(service=servico)
# for list_route in lista_rotas:
#     sleep(3)
#     url = list_route[0]
#     navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
#     navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
# navegador.close()

In [ ]:
# pip install pyspark
from pyspark.sql import SparkSession
# Criando as variáveis que serão utilizadas no Spark
appname = 'tcc-project'
master = 'local'
# Criando a sessão Spark
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

In [ ]:
# Criando a sessão Spark
appname = 'tcc-project'
master = 'local'
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

In [ ]:
spark

In [ ]:
# pip install pyspark
from pyspark.sql import SparkSession
# Criando as variáveis que serão utilizadas no Spark
appname = 'tcc-project'
master = 'local'
# Criando a sessão Spark
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

spark
# Coalesce o DataFrame para um único arquivo
df_single_file = df.coalesce(1)
# Salve o DataFrame coalesced como um único arquivo CSV
df_single_file.write.csv(csv_caminho_arquivo, header=True, mode="overwrite")
Dessa forma o spark gera um csv particionado
df.write.csv('pyspark_dataframe.csv', header=True, mode="overwrite")